# Moneta Agents Collaboration Sample

Illustrates how `AgentGroupChat` can be used to orchestrate Azure AI Agent Service Agents for collaboratively solving problems.

This sample requires the setup of https://github.com/Azure-Samples/moneta-agents in Azure and the tools/Settings.cs adapted accordingly.

In [1]:
#r "nuget: Azure.Identity, 1.13.2"
#r "nuget: Azure.AI.Projects, 1.0.0-beta.4"
#r "nuget: Microsoft.SemanticKernel, 1.41.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.41.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.41.0-preview"
#r "nuget: Microsoft.SemanticKernel.Agents.AzureAI, 1.41.0-preview"
#r "nuget: Microsoft.SemanticKernel.Prompty, 1.41.0-alpha"
#r "nuget: Microsoft.Azure.Cosmos, 3.47.2"
#r "nuget: Microsoft.Extensions.Logging, 9.0.3"
#r "nuget: Microsoft.Extensions.Logging.Console, 9.0.3"
#r "nuget: Newtonsoft.Json, 13.0.3"
using Azure.Identity;
using Azure.AI.Inference;
using Azure.AI.Projects;
using Azure.AI.OpenAI;
using Microsoft.Azure.Cosmos;
using Microsoft.DotNet.Interactive;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.AzureAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Services;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Console;
using Microsoft.Extensions.Options;
using System.ComponentModel;
using System.Text.Json;
using System.Threading;
using System.Linq;
using Kernel = Microsoft.SemanticKernel.Kernel;
#!import tools/Settings.cs
#!import tools/Extensions.cs
#!import tools/NotebookLogger.cs
#!import tools/CrmClient.cs
#!import tools/OutputHelper.csx

Installed Packages Azure.AI.Projects, 1.0.0-beta.4 Azure.Identity, 1.13.2 Microsoft.Azure.Cosmos, 3.47.2 Microsoft.Extensions.Logging, 9.0.3 Microsoft.Extensions.Logging.Console, 9.0.3 Microsoft.SemanticKernel, 1.41.0 Microsoft.SemanticKernel.Agents.AzureAI, 1.41.0-preview Microsoft.SemanticKernel.Agents.Core, 1.41.0-preview Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.41.0 Microsoft.SemanticKernel.Prompty, 1.41.0-alpha Newtonsoft.Json, 13.0.3

Installed Packages Markdig, 0.40.0

In [2]:
#pragma warning disable SKEXP0070
var projectClient = new AIProjectClient(Settings.AIFoundryProjectConnectionString, new DefaultAzureCredential());
var agentsClient = projectClient.GetAgentsClient();
var serchConnection = (await projectClient.GetConnectionsClient().GetConnectionAsync("search-service-connection")).Value;
var aoaiConnection = (await projectClient.GetConnectionsClient().GetConnectionAsync("aoai-connection")).Value;

var kernelBuilder = Kernel.CreateBuilder();
kernelBuilder.Services.AddLogging(builder =>
{
    builder.SetMinimumLevel(LogLevel.Warning)
        .AddFilter("Microsoft.SemanticKernel.Agents.AgentGroupChat", LogLevel.Information)
        .AddFilter("Microsoft.SemanticKernel.Agents.Chat", LogLevel.Information)
        .AddSimpleConsole(o => o.SingleLine = false)
        .ClearProviders()
        .Services.AddSingleton<ILoggerProvider, NotebookLoggerProvider>();
});
kernelBuilder.Services.AddAzureOpenAIChatCompletion(Settings.ModelDeploymentName, aoaiConnection.Properties.Target, new DefaultAzureCredential());
var defaultKernel = kernelBuilder.Build();

### Create the agents in Azure AI and then the corresponding Semantic Kernel agent objects

In [3]:
/* CIO Agent - Looks up documents from CIO (Chief Investment Office) for strategies and in-house view from Moneta Bank. */
#pragma warning disable SKEXP0110
var cioAgentDefinition = await agentsClient.CreateOrUpdateAgentAsync(
    model: Settings.ModelDeploymentName,
    name: "CIOAgent",
    description: "Agent processes the user request about investement researches and in house views from documents produced by the CIO (Chief Investment Office) and in-house view from Moneta Bank.",
    instructions:
        """
        You are an assistant that responds to the user query about investement researches and in house views from documents produced by the CIO (Chief Investment Office) and in-house view from Moneta Bank.

        **Your Responsibilities:**
        - Provide information about researches and investement reccomendations from documents produced by the CIO (Chief Investment Office) and in-house view from Moneta Bank by using the provided function: 'search_cio'.
        - Don't use your general knowledge to respond. Use only the provided functions.
        - Provide CONCISE and specific answer to the user's question. Do not provide general information.
        - Make sure to provide accurate and relevant information based on the user's inquiry.
        """,
    temperature: 0.1f,
    tools: [new AzureAISearchToolDefinition()],
    toolResources: new ToolResources(){
        AzureAISearch = new AzureAISearchResource(){
            IndexList = {
                new IndexResource(serchConnection.Id, "cio-index"),
            }
        }
    }
);
var cioAgent = new AzureAIAgent(cioAgentDefinition, agentsClient){
    Kernel = defaultKernel.Clone()
};

Agent with name 'CIOAgent' already exists. Updating the existing agent.


In [4]:
/* CRM CLient - Uses CosmosDB to look up customer data */
#pragma warning disable SKEXP0110
var crmPlugin = KernelPluginFactory.CreateFromType<CrmClient>();
var crmAgentDefinition = await agentsClient.CreateOrUpdateAgentAsync(
    model: Settings.ModelDeploymentName,
    name: "CRMAgent",
    description: "Agent processes the user request about investement researches and in house views from documents produced by the CIO (Chief Investment Office) and in-house view from Moneta Bank.",
    instructions:
        """
        You are an assistant that responds to the user query about client data and client portfolios.

        **Your Task:**
        - FIRST carefully check if the customer name or client id is mentioned in the user request.
        - If the request contains client ID or customer name then use CRM functions to retrieve customer policy details, 'load_from_crm_by_client_fullname' or 'load_from_crm_by_client_id' accordingly.
        - DO NOT ask for the client's name or id. If you receive a request that references a customer without providing ID or Name, don't provide an answer and terminate. 
        - Don't use your general knowledge to respond. Use only the provided functions.
        - Provide CONCISE and specific answer to the user's question. Do not provide general information.
        - Make sure to provide accurate and relevant information based on the user's inquiry.
        """,
    temperature: 0.1f,
    tools: crmPlugin.Select(func => new FunctionToolDefinition(func.Metadata.Name, func.Metadata.Description)).ToList()
);
var crmKernel = defaultKernel.Clone();
crmKernel.Plugins.Add(crmPlugin);
var crmAgent = new AzureAIAgent(crmAgentDefinition, agentsClient) {
    Kernel = crmKernel
};

Agent with name 'CRMAgent' already exists. Updating the existing agent.


In [5]:
/* Funds Agent - processes the user request about generic funds and ETFs information  */
/* for this demo we look up data from an AI Search index */
#pragma warning disable SKEXP0110
var fundsAgentDefinition = await agentsClient.CreateOrUpdateAgentAsync(
    model: Settings.ModelDeploymentName,
    name: "FundsAgent",
    description: "Agent processes the user request about generic funds and ETFs information by calling external data sources",
    instructions:
        """
        You are an assistant that responds to the user query about generic funds and ETFs information.

        **Your Task:**
        - Provide information about funds, etfs,  etc. offered by using the provided function: 'search_funds_details'.
        - When using 'search_funds_details' function YOU MUST include all details that are relevant to the user's inquiry - such as funds top holdings, sector exposures, perfomances, etc.
        - When using 'search_funds_details' function YOU MUST include user question AS IS
        - Don't use your general knowledge to respond. Use only the provided functions.
        - Provide CONCISE and specific answer to the user's question. Do not provide general information.
        - Make sure to provide accurate and relevant information based on the user's inquiry.
        """,
    temperature: 0.1f,
    tools: [new AzureAISearchToolDefinition()],
    toolResources: new ToolResources(){
        AzureAISearch = new AzureAISearchResource(){
            IndexList = {
                new IndexResource(serchConnection.Id, "funds-index"),
            }
        }
    }
);
var fundsAgent = new AzureAIAgent(fundsAgentDefinition, agentsClient){
    Kernel = defaultKernel.Clone()
};

Agent with name 'FundsAgent' already exists. Updating the existing agent.


In [6]:
/* Funds Agent - processes the user request about generic funds and ETFs information  */
/* for this demo we look up data from an AI Search index */
#pragma warning disable SKEXP0110
var summarizerAgentDefinition = await agentsClient.CreateOrUpdateAgentAsync(
    model: Settings.ModelDeploymentName,
    name: "SummariserAgent",
    description: "The agent that summarises a final response to the user's original inquiry by collecting responses from the OTHER agentsurces",
    instructions:
        """
        You are a Summariser Agent. You receive information from multiple agents and need to compile a final response to the user.\n

        Provide a coherent and helpful response to the user by combining the information from the agents. \n
        - You **MUST** use only information provided in the chat history. Do NOT use any external sources or functions. \n
        - You **MUST** assume that the information provided by other agents is not visible to the end user. \n
        - You **MUST** formulate the full answer based on the information from other agents. \n
        - You **MUST** check the original USER question, scan the chat history, and summarise the coherent reponse \n

        The response MUST NOT be the original question, or generic statement. 

        It **MUST** be specific to the user's inquiry and provide a clear and concise answer.

        FINAL RESPONSE:
        """,
    temperature: 0.7f
);
var summarizerAgent = new AzureAIAgent(summarizerAgentDefinition, agentsClient){
    Kernel = defaultKernel.Clone()
};

Agent with name 'SummariserAgent' already exists. Updating the existing agent.


### Create the Agent Group Chat object.

This sets up an `AgentGroupChat` with the agents and uses a custom speaker selection strategy which at the und uses the summarizer agent to generate the final response.

In [7]:
#pragma warning disable SKEXP0110

Microsoft.SemanticKernel.Agents.Agent[] agents = [cioAgent, crmAgent, fundsAgent, summarizerAgent];

var definitions = string.Join("\n",
    agents.Select(agent => $"{agent.Name}: {agent.Description}"));
var speakerSelectionPrompt =
    $$$"""
    You are the next speaker selector.

    - You MUST return ONLY agent name from the list of available agents below.
    - You MUST return the agent name and nothing else.
    - Check the history, if any, and decide WHAT agent is the best next speaker
    - The names are case-sensitive and should not be abbreviated or changed.
    - YOU MUST OBSERVE AGENT USAGE INSTRUCTIONS.

    # AVAILABLE AGENTS

    {{{definitions}}}

    # CHAT HISTORY

    {{$history}}
    """;

var groupChatStrategyKernel = defaultKernel.Clone();
var speakerSelectorFunction = groupChatStrategyKernel.CreateFunctionFromPrompt(
    functionName: "SpeakerSelector",
    promptTemplate: speakerSelectionPrompt,
    executionSettings: new AzureOpenAIPromptExecutionSettings() {
    Temperature = 0.0f,
    MaxTokens = 50,
});

class AlwaysTrueTerminationStrategy : TerminationStrategy
{
    protected override Task<bool> ShouldAgentTerminateAsync(Microsoft.SemanticKernel.Agents.Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken = default)
    {
        // Always return true to terminate the conversation, this assumes that the last agent or limit is set correctly
        return Task.FromResult(true);
    }
}

AgentGroupChat CreateAgentGroupChat() => new AgentGroupChat(agents)
{
    ExecutionSettings = new(){
        SelectionStrategy = new KernelFunctionSelectionStrategy(speakerSelectorFunction, groupChatStrategyKernel) {
            ResultParser = (result) =>
                result.GetValue<string>() is { Length: > 0 } agentName
                    ? agents.FirstOrDefault(a => a.Name == agentName).Name ?? summarizerAgent.Name
                    : summarizerAgent.Name,
            AgentsVariableName = "agents",
            HistoryVariableName = "history",
        },
        // Terminate when the summarizer agent was selected last or the maximum number of iterations is reached
        TerminationStrategy = new AlwaysTrueTerminationStrategy() {
            Agents = [summarizerAgent],
            MaximumIterations = 10
        }
    },
    LoggerFactory = defaultKernel.LoggerFactory
};

### Execute the Group chat using a staring prompt

In [8]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

var input = new ChatMessageContent(AuthorRole.User, "Craft a rebalance proposal for the client Pete Mitchell increasing the weight of investments in tech stocks absed on our offering");

var response =
    """
    To craft a rebalance proposal for Pete Mitchell focusing on increasing investments in tech stocks, the strategy should leverage Moneta Bank's insights into high-growth sectors:

    1. **Megacap Tech Companies**: These firms are positioned to thrive in the AI era, offering sustained earnings growth of 15-20% driven by AI monetization. Their dominant market position and robust free cash flow generation make them attractive for long-term investments, even amidst market volatility.

    2. **AI Ecosystem Investments**: Focusing on companies within the AI value chain, particularly in semiconductors, cloud infrastructure, and data centers, ensures exposure to one of the most compelling long-term growth opportunities. These areas are projected to see substantial capital investment and earnings growth.

    3. **Technology Sector Diversification**: Include technology companies with strong balance sheets and leadership in AI innovation. Diversify further by considering consumer brands with stable pricing power and healthcare firms capitalizing on structural trends.

    This proposal emphasizes quality growth stocks in technology, leveraging industry leadership and innovation to enhance portfolio returns while aligning with current market trends.
    """;

var chatUi = OutputHelper.RenderHtmlForConversation(input.Content, response).Display();

var groupChat = CreateAgentGroupChat();
try
{
    groupChat.AddChatMessage(input);

    string lastMessageContent = "sorry, unable to process your request";

    await foreach (var response in groupChat.InvokeAsync())
    {
        Console.WriteLine($"[{response.Role} - {response.AuthorName}] {response.Content}");
        lastMessageContent = response.Content;
    }
    
    Console.WriteLine($"\n[IS COMPLETED: {groupChat.IsComplete}]");

    chatUi.Update(
        OutputHelper.RenderHtmlForConversation(
            input.Content,
            lastMessageContent
        )
    );
}
finally
{
    await groupChat.ResetAsync();
}

Craft a rebalance proposal for the client Pete Mitchell increasing the weight of investments in tech stocks absed on our offering 

 
 
 
 
 To craft a tailored rebalance proposal for Pete Mitchell, the strategy should emphasize increasing exposure to high-growth technology stocks, particularly those driving advancements in AI and the semiconductor sector. Here are the recommendations: 
 
 Tech Sector Focus : 
 
 Invest heavily in megacap tech firms such as Apple, Microsoft, and NVIDIA, which dominate the AI innovation space and are projected to see strong earnings growth. 
 
 
 Semiconductor Investments : 
 
 Diversify into high-quality semiconductor companies specializing in AI chips, as this sector is expected to grow earnings by 50% this year. 
 
 
 AI Ecosystem : 
 
 Include stocks in cloud infrastructure and data centers which support AI growth, with annual investments forecasted to reach USD 331 billion by 2027. 
 
 
 ETF Option : 
 
 Consider the Moneta Bank MSCI World ETF, which allocates 22% to Information Technology and includes key holdings like Apple, Microsoft, and NVIDIA. The ETF provides diversification, cost efficiency (0.15% expense ratio), and liquidity, making it ideal for balancing growth and risk. 
 
 
 
 This strategy ensures Pete benefits from the robust growth potential in tech, while maintaining diversification and cost-effective asset management.

info: Microsoft.SemanticKernel.Agents.AgentGroupChat[0]
      [AddChatMessages] Added Messages: 1.

info: Microsoft.SemanticKernel.Agents.Chat.KernelFunctionSelectionStrategy[0]
      [NextAsync] Invoked function: (null).SpeakerSelector: Microsoft.SemanticKernel.Connectors.OpenAI.OpenAIChatMessageContent

info: Microsoft.SemanticKernel.Agents.AgentGroupChat[0]
      [InvokeAsync] Agent selected Microsoft.SemanticKernel.Agents.AzureAI.AzureAIAgent: asst_FE2zFmUZVvgR4oLs2fBuFX8R/CIOAgent by Microsoft.SemanticKernel.Agents.Chat.KernelFunctionSelectionStrategy

info: Microsoft.SemanticKernel.Agents.AgentGroupChat[0]
      [InvokeAgentAsync] Created channel for Microsoft.SemanticKernel.Agents.AzureAI.AzureAIAgent: asst_FE2zFmUZVvgR4oLs2fBuFX8R/CIOAgent

[assistant - CIOAgent] To craft a rebalance proposal for Pete Mitchell focusing on increasing the weight of investments in tech stocks, consider the following recommendations based on Moneta Bank's CIO insights:

1. **Focus on AI-Driven Grow